<a href="https://colab.research.google.com/github/sherxon/kaggle/blob/main/disaster_twiiter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Embedding, Activation, Flatten, LSTM, Dropout
from sklearn import feature_extraction
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [ ]:
train_df= pd.read_csv('/content/drive/My Drive/kaggle/disaster_twitter/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/kaggle/disaster_twitter/test.csv')

In [ ]:
#count_vectorizer = feature_extraction.text.CountVectorizer()

In [ ]:
t= Tokenizer()

In [ ]:
t.fit_on_texts(list(train_df['text'])+ list(test_df['text']))

In [ ]:
train_encoded = t.texts_to_sequences(train_df['text'])
test_encoded = t.texts_to_sequences(test_df['text'])

In [ ]:
max_length=0;
for tweet in train_encoded:
  if(max_length < len(tweet)):
    max_length=len(tweet)
print('max_length', max_length)

max_length 33


In [ ]:
padded_tweets=pad_sequences(train_encoded, maxlen=max_length, padding='post', value=0.0)
padded_test = pad_sequences(test_encoded, maxlen=max_length, padding='post', value=0.0)
print('train_padded', padded_tweets.shape)
print('test_padded', padded_test.shape)

train_padded (7613, 33)
test_padded (3263, 33)


In [ ]:
valid_words = t.word_index.keys()

In [ ]:
embeddings_index = dict()
f = open('glove.6B.50d.txt')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  if word in valid_words:
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 15001 word vectors.


In [ ]:
vocab_size = len(t.word_index)+1
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(31127, 50)

In [ ]:
#train_vectors = count_vectorizer.fit_transform(train_df['text'])
#test_vectors = count_vectorizer.transform(test_df['text'])
#print(train_vectors.shape)
#print(test_vectors.shape)

In [ ]:
Y=train_df['target'].to_numpy()
print(Y.shape)

(7613,)


In [ ]:
tweets_count=padded_tweets.shape[0]
print('vocab_size', vocab_size)
print('shape of train_input', padded_tweets.shape)
print('input tweet count', tweets_count)

vocab_size 31127
shape of train_input (7613, 33)
input tweet count 7613


In [ ]:
input = Input(shape=(max_length,))
embeddings=Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=max_length, trainable=False)(input)
X=LSTM(128, return_sequences=True)(embeddings)
X=Dropout(0.3)(X)
X=LSTM(128, return_sequences=False)(X)
X=Dropout(0.3)(X)
X=Dense(8, 'relu')(X)
X=Dense(1, activation='sigmoid')(X)
model = Model(input, X);
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 33, 50)            1556350   
_________________________________________________________________
lstm_4 (LSTM)                (None, 33, 128)           91648     
_________________________________________________________________
dropout_4 (Dropout)          (None, 33, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                

In [ ]:
model.compile('Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(padded_tweets, Y,  epochs = 20, batch_size = 64, shuffle = True)

Epoch 1/20
119/119 [==============================] - 16s 133ms/step - loss: 0.5062 - accuracy: 0.7645
Epoch 2/20
119/119 [==============================] - 16s 132ms/step - loss: 0.4565 - accuracy: 0.7973
Epoch 3/20
119/119 [==============================] - 20s 171ms/step - loss: 0.4313 - accuracy: 0.8087
Epoch 4/20
119/119 [==============================] - 16s 133ms/step - loss: 0.4237 - accuracy: 0.8102
Epoch 5/20
119/119 [==============================] - 16s 134ms/step - loss: 0.4123 - accuracy: 0.8200
Epoch 6/20
119/119 [==============================] - 16s 133ms/step - loss: 0.3993 - accuracy: 0.8285
Epoch 7/20
119/119 [==============================] - 16s 133ms/step - loss: 0.3973 - accuracy: 0.8296
Epoch 8/20
119/119 [==============================] - 16s 133ms/step - loss: 0.3813 - accuracy: 0.8337
Epoch 9/20
119/119 [==============================] - 16s 134ms/step - loss: 0.3848 - accuracy: 0.8349
Epoch 10/20
119/119 [==============================] - 16s 134ms/step - l

In [ ]:
prediction = model.predict(padded_test)

In [ ]:
sample_suggestion=pd.read_csv('/content/drive/My Drive/kaggle/disaster_twitter/sample_submission.csv')

In [ ]:
prediction=np.rint(prediction).astype(int)

In [ ]:
sample_suggestion['target']=prediction
sample_suggestion.head()

,id,target
0,0,1
1,2,1
2,3,0
3,9,1
4,11,1


In [ ]:
sample_suggestion.to_csv('/content/drive/My Drive/kaggle/disaster_twitter/prediction5.csv', index=False)